# Now we'll bring in the model and run it

In [ ]:
# Import libraries

%matplotlib inline
import os
import flopy
import pyemu
import pandas as pd
import numpy as np
import subprocess as sp
from pyemu.pst.pst_utils import SFMT

### Specify a path to the model folder by Users

In [ ]:
wd = "D:/Workshops/qsm_mb/SWAT-MODFLOW"
os.chdir(wd)
mname = "modflow.mfn"

### Check MODFLOW model

In [ ]:
m = flopy.modflow.Modflow.load(mname,model_ws=wd)
m.check()

# 1. Let's run the model first to get pre-calibration results

- Simulation Period 
    * Jan. 01, 1995 ~ Dec. 31, 1995 with no warm-up period (1 year)
    <br/><br/>
- Measurement Duration
    * Streamflow
        - Outlet: Reach 58 Jan. 1, 1993 ~ Dec. 31, 2012
    * Watertable
        - Grid ID: 5699, 5832


#### Example of Setup
- Calibration / Validation
    * 2003 1/1 - 2007 for Calibration
    * 2008 - 2010 for validation (???)
    <br/><br/>
- Streamflow: Little baseflow, High peak, Peak early -> , peak low, shift to little right
    - SWAT parameters:
        * Little Baseflow & High Peak
            - CN2 - Decrease
            - ESCO - Increase
            - SOL_AWC - Increase
        * Peak early
            * HRU_SLP - Decrease
            * OV_N - Increase
            * SLSUBBSN - Increase (The value of overland flow length)
    - MODFLOW parameters:
        * Little Baseflow & High Peak
            - Riverbed cond - Decrease
            - Riverbed bottom elevation - Decrease
        * K - ? 
        * Sy - ?
        
- Watertable -> high watertable, slow recession, 
    - SWAT parameters:
        * RCHRG_DP - (turned off) decrease
        * CN2 - ?
        * ESCO - ? (increase)
        * SOL_AWC - decrease
    - MODFLOW parameters:
        * K - increase 
        * Sy - ?
        * EVT depth - increase
        * River Bottom - ?
        * River conductance - decrease

### Import additional libraries to work with SWAT-MODFLOW inputs / ouputs

In [ ]:
from sm_pst_pkgs import sm_pst_par, sm_pst_utils, sm_pst_stats

# 2. Create PEST input files (template / instruction)
## 2.1. Create template files
We are going to use the `*.pval` and `mf_river.par` files for MODFLOW parameters and `model.in` file for SWAT parameters.

In [ ]:
# pval file
pval_file = 'mf_300.pval'
# model.in file
model_in = 'model.in'

In [ ]:
gw_par = pyemu.utils.gw_utils.modflow_pval_to_template_file(pval_file, tpl_file=None)
gw_par

In [ ]:
sm_pst_utils.model_in_to_template_file?

In [ ]:
sw_par = sm_pst_utils.model_in_to_template_file(model_in, tpl_file=None)
sw_par

## 2.2 Parameterize Riverbed conductance and thickness

In [ ]:
sm_pst_par.create_riv_par?

In [ ]:
# provide channel ids that will be used for calibration
subs = ['g1', 'g2', 'g3', 'g4', 'g5']
sm_pst_par.create_riv_par(wd, subs)

In [ ]:
# create a template file for mf_riv.par file
sm_pst_utils.riv_par_to_template_file('mf_riv.par')

In [ ]:
# overwrite the river package file
sm_pst_par.riv_par(wd)

# 3. Build instruction files (streamflow / watertable)
## 3.1 Extract simulated streamflow data from (output.rch)

In [ ]:
# file path
rch_file = 'output.rch'
# reach numbers that are used for calibration
subs = [58]

In [ ]:
# extract month_streamflow
sm_pst_utils.extract_day_str(rch_file, subs, '1/1/1995', '1/1/1995', '12/31/1995')

### 3.1.1 Create instruction files for each str_sim file using the 'streamflow.obd' file

In [ ]:
sim_files = ['cha_{:03d}.txt'.format(x) for x in subs]
sim_files

In [ ]:
col_names = ['ch58']

In [ ]:
# create instruction files for each sim file
for i in range(len(sim_files)):
    sm_pst_utils.str_obd_to_ins(sim_files[i], col_names[i], '1/1/1995', '12/31/1995')

## 3.2 Extract simulated watertable from (swatmf_out_MF_obs)

In [ ]:
sm_pst_utils.extract_watertable_sim([5699, 5832], '1/1/1995', '12/31/1995')


In [ ]:
sm_pst_utils.mf_obd_to_ins('wt_5832.txt', 'g_5832', '1/1/1995', '12/31/1995')

#### Let's try to create the second instruction file for the 'g_5699'

<details>
<summary>Hint!</summary>    
sm_pst_utils.mf_obd_to_ins('wt_5699.txt', 'g_5699', '1/1/1995', '12/31/1995')
</details>

# 4. Create a pst (control) file 

##### Let's create a dummy pst file based on tpl and ins files we've created

In [ ]:
io_files = pyemu.helpers.parse_dir_for_io_files('.')
pst = pyemu.Pst.from_io_files(*io_files)
pyemu.helpers.pst_from_io_files(io_files[0], io_files[1], io_files[2], io_files[3], 'sm_pest_dummy.pst')

##### We need to modify parameter ranges and offset setting.

In [ ]:
par = pst.parameter_data
par

## 4.1 Modify a default parameter setting

### 4.1.1 Change parameter group name

In [ ]:
for i in range(len(par)):
    if (par.iloc[i, 0][:2]) == 'sy':
        par.iloc[i, 6] = 'sy'
    elif par.iloc[i, 0][:7] == 'rivbot_':
        par.iloc[i, 6] = 'rivbot'
    elif par.iloc[i, 0][:6] == 'rivcd_':
        par.iloc[i, 6] = 'rivcd'
    elif par.iloc[i, 0][:2] == 'hk':
        par.iloc[i, 6] = 'hk'
    else:
        par.iloc[i, 6] = 'str'
print(par)

### 4.1.2 Change MODFLOW default values

In [ ]:
count = 0
for i in range(len(par)):
    if (par.iloc[i, 6] == 'hk'):
        par.iloc[i, 3] = 1  
        par.iloc[i, 4] = 1.000000e-02
        par.iloc[i, 5] = 1.000000e+02
    elif (par.iloc[i, 6] == 'sy'):
        par.iloc[i, 3] = 1.000000e-02       
        par.iloc[i, 4] = 1.000000e-04
        par.iloc[i, 5] = 0.6  
    elif (par.iloc[i, 6] == 'rivbot'):
        par.iloc[i, 3] = 3.001     
        par.iloc[i, 4] = 0.001
        par.iloc[i, 5] = 6
        par.iloc[i, 8] = -3
    elif (par.iloc[i, 6] == 'rivcd'):
        par.iloc[i, 3] = 50.001       
        par.iloc[i, 4] = 0.001
        par.iloc[i, 5] = 100
        par.iloc[i, 8] = -50
    else:
        count += 1
count

### 4.1.3 Change SWAT default values

In [ ]:
par

In [ ]:
# CN2
par.loc['cn2', 'parval1'] = 1.001
par.loc['cn2', 'parlbnd'] = 0.8
par.loc['cn2', 'parubnd'] = 1.2
par.loc['cn2', 'offset'] = -1

# CH_K2
par.loc['ch_k2', 'parval1'] = 0.001
par.loc['ch_k2', 'parlbnd'] = 0.0001
par.loc['ch_k2', 'parubnd'] = 10

# CH_S2
par.loc['ch_s2', 'parval1'] = 1.0001
par.loc['ch_s2', 'parlbnd'] = 0.001
par.loc['ch_s2', 'parubnd'] = 1.5
par.loc['ch_s2', 'offset'] = -1


# # ESCO
# par.loc['esco', 'parval1'] = 1.001
# par.loc['esco', 'parlbnd'] = 0.5
# par.loc['esco', 'parubnd'] = 1.5
# par.loc['esco', 'offset'] = -1

# # sol_awc()
# par.loc['sol_awc()', 'parval1'] = 1.001
# par.loc['sol_awc()', 'parlbnd'] = 0.5
# par.loc['sol_awc()', 'parubnd'] = 1.5
# par.loc['sol_awc()', 'offset'] = -1



In [ ]:
par

## 4.2 Modify a default observation setting

In [ ]:
# set observation group
obd = pst.observation_data
obd

In [ ]:
# Change obd group name
for i in range(len(obd)):
    obd.iloc[i, 3] = obd.iloc[i, 0][:-7]
obd

## 4.3 Import measured data

In [ ]:
# Streamflow
stf_obd = pd.read_csv('streamflow.obd',
                       sep='\t',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
stf_obd = stf_obd['1/1/1995': '12/31/1995']
stf_obd

In [ ]:
# watertable
wt_obd = pd.read_csv('modflow.obd',
                       sep='\t',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
wt_obd = wt_obd['1/1/1995': '12/31/1995']
wt_obd

In [ ]:
# Get sub list based on obd order
obd_order = []
for i in obd.obgnme.tolist():
    if i not in obd_order:
        obd_order.append(i)
obd_order

In [ ]:
# get total list from each sub obd, delete na vals
tot_obd = []
for i in obd_order[:1]:
    tot_obd += stf_obd[i].dropna().tolist()
for i in obd_order[1:]:
    tot_obd += wt_obd[i].dropna().tolist()
len(tot_obd)

In [ ]:
obd.loc[:, 'obsval'] = tot_obd
obd

# 5. Export the final control file

In [ ]:
pst.write('mb_pest.pst')